In [29]:
# This code is written by Yara Al-Shorman
# Date created: May 18 2022
# Last modified: May 22 2022
# Github repo: https://github.com/YaraAlShorman/Research-spring-22
# The purpose of this code is to predict the bulkflows for a large given set of galaxies (using machine learning)

### imports

In [43]:
# imports
import tensorflow as tf
from tensorflow_datasets import load as tfds_load
from numpy import load, concatenate, expand_dims  # I am doing this to minimize unncessary imports, they take up too much RAM and time
from sklearn.preprocessing import MinMaxScaler
from math import floor
# import matplotlib.pyplot as plt

### data preprocessing

In [40]:
# stacking all of the data into one humungous tensor
# I am using tensors because they can be ragged and can be stacked
# these means we won't have to cut off our data to standardize array sizes
data = load(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-0-error-40.npy.npz')
input_data = data['data']
scaler = MinMaxScaler()
scaler.fit(input_data)
input_data = scaler.transform(input_data)
input_data = tf.expand_dims(input_data, axis=0)
input_data = tf.RaggedTensor.from_tensor(input_data)
# if scaling the data this way doesn't work good enough, i can try scaling all the data at once, to one scale
# in this case, i am scaling every 'square' of data seprately
# i should try scaling the entire 'cube' at once

output_data = tf.convert_to_tensor(data['header'])
output_data = tf.expand_dims(output_data, axis=0)

num_of_files = 10  # variable for number of files to load, current limit is 3000

for i in range(1, num_of_files): # limit 3000
        array_data = load(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-{i}-error-40.npy.npz')
        # 'data' data
        temp = tf.convert_to_tensor(array_data['data'], dtype='float64')
        scaler = MinMaxScaler()
        scaler.fit(temp)
        temp = scaler.transform(temp)
        temp = tf.expand_dims(temp, axis=0)  # dimensions have to be expandad to be able to concat along the outer-dimension 
        input_data = tf.concat([input_data, temp], axis=0)  # concat adds to existing dimensions, does not create new ones
        # 'input_data' is a tensor
        
        # 'header' data (bulkflows)
        temp = array_data['header']
        temp = expand_dims(temp, axis=0)
        output_data = concatenate([output_data, temp], axis=0)
        # 'output_data' is an array, because it is uniform
        # i scale it (normalize it) all at once, then i covnert it to a tensor
        
# next is scaling

In [65]:
# normalizing/scaling the data
scaler = MinMaxScaler()
scaler.fit(output_data)
output_data = scaler.transform(output_data)
output_data = tf.convert_to_tensor(output_data)

In [66]:
# splitting the data into train/test sections
# 75% -> train
# 25% -> test
train_percent = 0.75
input_train, input_test = (input_data[:floor(num_of_files * train_percent)], \
                                input_data[floor(num_of_files * train_percent)::])
output_train, output_test = (output_data[:floor(num_of_files * train_percent)], \
                                output_data[floor(num_of_files * train_percent)::])

# data is split and ready for the model

### model creation

In [67]:
# optimizer

optimizer = tf.keras.optimizers.Adam()

In [80]:
# model creations

model0 = tf.keras.Sequential()
model0.add(tf.keras.Input(shape=(None, 4), ragged=False))      
model0.add(tf.keras.layers.Dense(int(num_of_files / 2), activation='relu'))    
model0.add(tf.keras.layers.Dropout(0.5))
model0.add(tf.keras.layers.Dense(int(num_of_files / 4), activation='relu'))
model0.add(tf.keras.layers.Dropout(0.5))
model0.add(tf.keras.layers.Dense(128, activation='relu'))
model0.add(tf.keras.layers.Dropout(0.5))
model0.add(tf.keras.layers.Dense(32, activation='relu'))
model0.add(tf.keras.layers.Dropout(0.5))
model0.add(tf.keras.layers.Dense(3, activation='linear'))

model0.compile(loss='mean_squared_error', optimizer=optimizer)
model0.fit(input_train, output_train, epochs=20, batch_size=num_of_files, use_multiprocessing=True, workers=4)


# epoch_step = 4  # epochs will go in sets of 4 before changing 
# epoch_lower = 0  # initial 
# epoch_upper = epoch_step  # initial 
# while epoch_upper < 21:  # adding on 20 epochs 
#     a = model0.fit(input_train, output_train, initial_epoch=epoch_lower, epochs=epoch_upper, batch_size=num_of_files, use_multiprocessing=True)
#     print(a.history['loss'])  # this is how  you get the loss when the model above is fit with the given lr
#     epoch_lower += epoch_step
#     epoch_upper += epoch_step  


Epoch 1/20


C:\Users\yaras\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/mean_squared_error/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/mean_squared_error/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 3), dtype=float32), dense_shape=Tensor("gradient_tape/mean_squared_error/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1264.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1264.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\yaras\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\yaras\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "